### 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [1]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    def __str__(self):
        return self.surface

In [7]:
import re
sentences = []
sentence = []
for line in open('data/neko.txt.cabocha'):
    x = re.search('(?P<surface>[^\t]*?)\t(?P<pos>[^,]*),(?P<pos1>[^,]*),[^,]*,[^,]*,[^,]*,[^,]*,(?P<base>[^,]*).*',line)
    if x:
        morph = Morph(x.group('surface'), x.group('base'), x.group('pos'), x.group('pos1'))
        sentence.append(morph)
    elif line == 'EOS\n' and not sentence == []:
        sentences.append(sentence)
        sentence = []

In [8]:
print([[str(y) for y in x] for x in sentences[:10]])

[['一'], ['\u3000', '吾輩', 'は', '猫', 'で', 'ある', '。'], ['名前', 'は', 'まだ', '無い', '。'], ['\u3000', 'どこ', 'で', '生れ', 'た', 'か', 'とんと', '見当', 'が', 'つか', 'ぬ', '。'], ['何', 'でも', '薄暗い', 'じめじめ', 'し', 'た', '所', 'で', 'ニャーニャー', '泣い', 'て', 'いた事', 'だけ', 'は', '記憶', 'し', 'て', 'いる', '。'], ['吾輩', 'は', 'ここ', 'で', '始め', 'て', '人間', 'という', 'もの', 'を', '見', 'た', '。'], ['しかも', 'あと', 'で', '聞く', 'と', 'それ', 'は', '書生', 'という', '人間', '中', 'で', '一番', '獰悪', 'な', '種族', 'で', 'あっ', 'た', 'そう', 'だ', '。'], ['この', '書生', 'という', 'の', 'は', '時々', '我々', 'を', '捕え', 'て', '煮', 'て', '食う', 'という', '話', 'で', 'ある', '。'], ['しかし', 'その', '当時', 'は', '何', 'という', '考', 'も', 'なかっ', 'た', 'から', '別段', '恐し', 'いとも', '思わ', 'なかっ', 'た', '。'], ['ただ', '彼', 'の', '掌', 'に', '載せ', 'られ', 'て', 'スー', 'と', '持ち上げ', 'られ', 'た', '時', '何だか', 'フワフワ', 'し', 'た', '感じ', 'が', 'あっ', 'た', 'ばかり', 'で', 'ある', '。']]


In [9]:
print([str(x) for x in sentences[2]])

['名前', 'は', 'まだ', '無い', '。']


### 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [10]:
class Chunk:
    def __init__(self, morphs, dst, srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs
    def __str__(self):
        ret = ''.join([str(x) for x in self.morphs])
        return ret

In [12]:
import re
sentences = []#sentence の list
sentence = []#Chunk の list
for line in open('data/neko.txt.cabocha'):
    x = re.search('(?P<surface>.*?)\t(?P<pos>[^,]*),(?P<pos1>[^,]*),[^,]*,[^,]*,[^,]*,[^,]*,(?P<base>[^,]*).*',line)
    if x:
        morph = Morph(x.group('surface'), x.group('base'), x.group('pos'), x.group('pos1'))
        sentence[-1].morphs.append(morph)
        continue
    y = re.search('\*\ (?P<number>[0-9]+)\ (?P<dst>-?[0-9]*)D.*', line)
    if y:
        dst = y.group('dst')
        srcs = [i for i, j in enumerate(sentence) if j.dst == y.group('number')]
        chunk = Chunk([], dst, srcs)
        sentence.append(chunk)
    elif line == 'EOS\n' and not sentence == []:
        sentences.append(sentence)
        sentence = []

In [13]:
s = sentences[7]
print([str(i) + '->' + str(s[int(i.dst)])  if i.dst != '-1' else str(i) for i in s])

['この->書生というのは', '書生というのは->話である。', '時々->捕えて', '我々を->捕えて', '捕えて->煮て', '煮て->食うという', '食うという->話である。', '話である。']


### 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [14]:
with open('data/file42', 'w'):
    pass
def st(s):
    return str(s).strip('、。「」')
with open('data/file42', 'a') as a:
    for sentence in sentences:
        for chunk in sentence:
            p = [st(sentence[int(i)]) for i in chunk.srcs] + ['「'+st(chunk)+'」']
            p = p + [st(sentence[int(chunk.dst)])] if chunk.dst != '-1' else p
            a.write('\t'.join(p) +  '\n')

In [16]:
with open('data/file42', 'r') as r:
    print(''.join(r.readlines()[:10]))

「一」
「　」	猫である
「吾輩は」	猫である
　	吾輩は	「猫である」
「名前は」	無い
「まだ」	無い
名前は	まだ	「無い」
「　どこで」	生れたか
　どこで	「生れたか」	つかぬ
「とんと」	つかぬ



### 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [17]:
with open('data/file43', 'w'):
    pass
def st(s):
    return str(s).strip('、。「」')
with open('data/file43', 'a') as a:
    for sentence in sentences:
        for chunk in sentence:
            if not [0 for i in chunk.morphs if i.pos == '名詞']:
                continue
            chun2 = sentence[int(chunk.dst)]
            if not [0 for i in chun2.morphs if i.pos == '動詞']:
                continue
            a.write(st(chunk) + '\t' + st(chun2) + '\n')

In [18]:
with open('data/file43', 'r') as r:
    print(''.join(r.readlines()[:10]))

　どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
記憶している	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと



### 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

In [19]:
#とりあえず使ってみる
import networkx as nx
import pygraphviz
G = nx.Graph()
G.add_node("a", label="wow")
G.add_node("b", label="あわれ")
G.add_edge("a","b")
g = nx.nx_agraph.to_agraph(G)
g.draw('data/nx_test.pdf' ,format='pdf', prog='dot') 

In [20]:
s = sentences[4]
G = nx.Graph()
for chunk in s:
    G.add_node(chunk, label=str(chunk))
for chunk in s[:-1]:
    G.add_edge(chunk, s[int(chunk.dst)])
g = nx.nx_agraph.to_agraph(G)
g.draw(f"data/file44.pdf",format='pdf', prog='dot')

### 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

- 動詞を含む文節において，最左の動詞の基本形を述語とする
- 述語に係る助詞を格とする
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

    始める  で
    
    見る    は を

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

- コーパス中で頻出する述語と格パターンの組み合わせ
- 「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [104]:
with open('data/file45', 'w'):
    pass
with open('data/file45', 'a') as a:
    for sentence in sentences:
        for chunk in sentence:
            if chunk.morphs[-1].surface == '、':
                chunk.morphs = chunk.morphs[:-1]
            verbs = [i.base for i in chunk.morphs if i.pos == '動詞']
            if verbs != []:
                verb = verbs[0]
                particles =  [sentence[i].morphs[-1].base for i in list(map(int,chunk.srcs)) if sentence[i].morphs[-1].pos == '助詞']
                if particles != []:
                    a.write(verb + '\t' + ' '.join(particles) + '\n')

↓スペースで区切ったやつを考慮にいれていない。

In [107]:
!sort data/file45 | uniq -c | sort -r | head -n 10

 705 云う	と
 467 する	を
 343 思う	と
 206 ある	が
 203 なる	に
 195 する	に
 182 見る	て
 161 する	と
 118 する	が
 100 見る	を
sort: Broken pipe


In [105]:
with open('data/file45b', 'w'):
    pass
with open('data/file45b', 'a') as a:
    for sentence in sentences:
        for chunk in sentence:
            if chunk.morphs[-1].surface == '、':
                chunk.morphs = chunk.morphs[:-1]
            verbs = [i.base for i in chunk.morphs if i.pos == '動詞']
            if verbs != []:
                verb = verbs[0]
                particles =  [sentence[i].morphs[-1].base for i in list(map(int,chunk.srcs)) if sentence[i].morphs[-1].pos == '助詞']
                for particle in  particles:
                    a.write(verb + '\t' + particle + '\n')

In [106]:
!sort data/file45b | uniq -c | sort -r | head -n 10

1314 する	を
1037 云う	と
 778 する	に
 578 する	と
 577 する	は
 518 なる	に
 518 する	が
 502 ある	が
 458 する	て
 410 思う	と
sort: Broken pipe


In [115]:
!grep -x -e 'する.*' data/file45b | sort | uniq -c | sort -r

1314 する	を
 778 する	に
 578 する	と
 577 する	は
 518 する	が
 458 する	て
 228 する	から
 220 する	も
 216 する	で
  58 する	の
  55 する	か
  44 する	ば
  38 する	へ
  36 する	まで
  35 する	でも
  21 する	として
  18 する	ながら
  17 する	って
  16 する	ばかり
  16 する	さえ
  15 する	ので
  15 する	たり
  13 する	くらい
  11 する	において
  11 する	だって
  11 する	じゃ
  11 する	や
   9 する	ほど
   8 する	なんか
   8 する	とも
   8 する	ね
   7 する	によって
   7 する	について
   7 する	より
   7 する	のに
   7 する	だけ
   6 する	をもって
   5 する	んで
   4 する	ども
   4 する	こそ
   4 する	し
   3 する	にあたって
   3 する	と共に
   3 する	とか
   3 する	ちゃ
   3 する	すら
   3 する	よ
   3 する	さ
   2 する	に対して
   2 する	たって
   2 する	けれど
   2 する	だり
   2 する	わ
   2 する	ぜ
   1 する	に従って
   1 する	けれども
   1 する	ものの
   1 する	に対し
   1 する	なんて
   1 する	けども
   1 する	やら
   1 する	ねえ
   1 する	にて
   1 する	なり
   1 する	など
   1 する	つつ
   1 する	だに
   1 する	ずつ
   1 する	な


In [116]:
!grep -x -e '見る.*' data/file45b | sort | uniq -c | sort -r

 380 見る	て
 176 見る	を
  60 見る	は
  53 見る	から
  45 見る	と
  39 見る	が
  36 見る	で
  35 見る	に
  21 見る	も
   7 見る	か
   6 見る	ながら
   6 見る	ので
   6 見る	たり
   6 見る	ば
   4 見る	ばかり
   4 見る	まで
   2 見る	より
   2 見る	って
   2 見る	じゃ
   2 見る	の
   2 見る	ね
   1 見る	に従って
   1 見る	によって
   1 見る	なんか
   1 見る	として
   1 見る	んで
   1 見る	なあ
   1 見る	だけ
   1 見る	すら
   1 見る	さえ
   1 見る	へ
   1 見る	な
   1 見る	し


In [117]:
!grep -x -e '与える.*' data/file45b | sort | uniq -c | sort -r

  16 与える	を
  12 与える	に
   8 与える	は
   7 与える	て
   2 与える	も
   2 与える	と
   2 与える	が
   1 与える	けれども
   1 与える	として
   1 与える	ば
   1 与える	で
   1 与える	か


### 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

- 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
- 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

    始める  で      ここで

    見る    は を   吾輩は ものを

In [108]:
with open('data/file46', 'w'):
    pass
with open('data/file46', 'a') as a:
    for sentence in sentences:
        for chunk in sentence:
            if chunk.morphs[-1].surface == '、':
                chunk.morphs = chunk.morphs[:-1]
            verbs = [i.base for i in chunk.morphs if i.pos == '動詞']
            if verbs != []:
                verb = verbs[0]
                units = [sentence[i] for i in list(map(int,chunk.srcs)) if sentence[i].morphs[-1].pos == '助詞']
                particles = [i.morphs[-1].base for i in units]
                if particles != []:
                    a.write(verb + '\t' + ' '.join(particles) + '\t' + ' '.join([str(i) for i in units]) + '\n')

In [109]:
i = 0
for line in open('data/file46'):
    print(line, end='')
    i = i + 1
    if i == 10:
        break

生れる	で	　どこで
つく	か が	生れたか 見当が
泣く	で	所で
する	て は	泣いて いた事だけは
始める	で	ここで
見る	は を	吾輩は ものを
聞く	で	あとで
捕える	を	我々を
煮る	て	捕えて
食う	て	煮て


### 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

- 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
- 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
- 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
- 例えば「別段くるにも及ばんさと、主人は手紙に返事をする。」という文から，以下の出力が得られるはずである．

    返事をする      と に は        及ばんさと 手紙に 主人は

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

- コーパス中で頻出する述語（サ変接続名詞+を+動詞）
- コーパス中で頻出する述語と助詞パターン

In [123]:
with open('data/file47', 'w'):
    pass
with open('data/file47', 'a') as a:
    for sentence in sentences:
        for chunk in sentence:
            if chunk.morphs[-1].surface == '、':
                chunk.morphs = chunk.morphs[:-1]
            verbs = [i.base for i in chunk.morphs if i.pos == '動詞']
            if verbs != []:
                verb = verbs[0]
                sahen = [sentence[i] for i in list(map(int,chunk.srcs)) \
                         if sentence[i].morphs[-1].base == 'を' \
                         and sentence[i].morphs[-2].pos1 == 'サ変接続' ]
                if sahen == []:
                    continue
                units = [sentence[i] for i in list(map(int,chunk.srcs)) if sentence[i].morphs[-1].pos == '助詞' and not sentence[i] in sahen]
                particles = [i.morphs[-1].base for i in units]
                if particles != []:
                    a.write(str(sahen[0]) + verb + '\t' + ' '.join(particles) + '\t' + ' '.join([str(i) for i in units]) + '\n')

In [124]:
!cut -f1 data/file47 | sort | uniq -c | sort -r | head -n 10

  25 返事をする
  19 挨拶をする
  11 話をする
   8 質問をする
   7 喧嘩をする
   6 真似をする
   5 質問をかける
   5 相談をする
   5 昼寝をする
   4 御辞儀をする


In [125]:
with open('data/file47b', 'w'):
    pass
with open('data/file47b', 'a') as a:
    for sentence in sentences:
        for chunk in sentence:
            if chunk.morphs[-1].surface == '、':
                chunk.morphs = chunk.morphs[:-1]
            verbs = [i.base for i in chunk.morphs if i.pos == '動詞']
            if verbs != []:
                verb = verbs[0]
                sahen = [sentence[i] for i in list(map(int,chunk.srcs)) \
                         if sentence[i].morphs[-1].base == 'を' \
                         and sentence[i].morphs[-2].pos1 == 'サ変接続' ]
                if sahen == []:
                    continue
                units = [sentence[i] for i in list(map(int,chunk.srcs)) if sentence[i].morphs[-1].pos == '助詞' and not sentence[i] in sahen]
                particles = [i.morphs[-1].base for i in units]
                for unit in units:
                    a.write(str(sahen[0]) + verb + '\t' + unit.morphs[-1].base+ '\n')

In [126]:
!sort data/file47b | uniq -c | sort -r | head -n 10

  23 返事をする	と
  11 挨拶をする	と
   9 返事をする	は
   4 質問をかける	は
   4 質問をかける	と
   4 返事をする	から
   4 挨拶をする	から
   4 返事をする	に
   4 欠伸をする	て
   4 挨拶をする	は


### 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を"->"で連結する

「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

- 吾輩は -> 見た
- ここで -> 始めて -> 人間という -> ものを -> 見た
- 人間という -> ものを -> 見た
- ものを -> 見た

In [122]:
for sentence in sentences:
        for chunk in sentence:
            if chunk.morphs[-1].surface in '、。':
                chunk.morphs = chunk.morphs[:-1]

In [128]:
with open('data/file48', 'w'):
    pass
def dst(chunk, sentence):
    dest = int(chunk.dst)
    if dest == -1:
        return str(chunk) + '\n'
    else:
        return str(chunk) + ' -> ' + dst(sentence[dest], sentence)
with open('data/file48', 'a') as a:
    for sentence in sentences:
        for chunk in sentence:
            nouns = [i.base for i in chunk.morphs if i.pos == '名詞']
            if nouns != []:
                noun = nouns[0]
                a.write(dst(chunk, sentence))

In [130]:
i = 0
for line in open('data/file48'):
    print(line, end='')
    i = i + 1
    if i == 20:
        break

一
吾輩は -> 猫である
猫である
名前は -> 無い
　どこで -> 生れたか -> つかぬ
見当が -> つかぬ
何でも -> 薄暗い -> 所で -> 泣いて -> 記憶している
所で -> 泣いて -> 記憶している
ニャーニャー -> 泣いて -> 記憶している
いた事だけは -> 記憶している
記憶している
吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た
あとで -> 聞くと -> 種族であったそうだ
それは -> 種族であったそうだ
書生という -> 人間中で -> 種族であったそうだ
人間中で -> 種族であったそうだ
一番 -> 獰悪な -> 種族であったそうだ


### 49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を"->"で連結して表現する
- 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．

- 文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
- 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を"|"で連結して表示

例えば，「吾輩はここで始めて人間というものを見た。」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

- Xは | Yで -> 始めて -> 人間という -> ものを | 見た
- Xは | Yという -> ものを | 見た
- Xは | Yを | 見た
- Xで -> 始めて -> Y
- Xで -> 始めて -> 人間という -> Y
- Xという -> Y

In [182]:
with open('data/file49', 'w'):
    pass

def stri(chunk, nX, nY):
    if chunk == nX:
        return ''.join(['X' if i == [j for j in chunk.morphs if j.pos == '名詞'][0] else str(i) for i in chunk.morphs])
    elif chunk == nY:
        return ''.join(['Y' if i == [j for j in chunk.morphs if j.pos == '名詞'][0] else str(i) for i in chunk.morphs])
    else:
        return str(chunk)


def dst(chunk, sentence, nX, nY, end):
    dest = int(chunk.dst)
    if dest == -1 or int(sentence[dest].dst) == end:
        return stri(chunk, nX, nY)
    else:
        return stri(chunk, nX, nY) + ' -> ' + dst(sentence[dest], sentence, nX, nY, end)

def ancestor(a, b, sentence):
    destA = int(a.dst)
    if destA == -1:
        return False
    return sentence[destA] == b or ancestor(sentence[destA], b, sentence)

def cousin(a, b, sentence):
    destA = int(a.dst)
    if ancestor(sentence[destA], b, sentence) or destA == -1:
        return destA
    else:
        return cousin(sentence[destA], b, sentence) 
    
def xy(nX, nY, sentence):
    destX = int(nX.dst)
    destY = int(nY.dst)
    if ancestor(nX, nY, sentence):
        return dst(nX, sentence, nX, nY, sentence[destY]) + '\n'
    else:
        c = cousin(nX, nY, sentence)
        return  dst(nX, sentence, nX, nY, c)+'|' + dst(nY, sentence, nX, nY, c) + '|' +dst(sentence[c], sentence, nX, nY, sentence[-1]) + '\n'
    

with open('data/file49', 'a') as a:
    for sentence in sentences:
        nouns = []
        for chunk in sentence:
            if [i for i in chunk.morphs if i.pos == '名詞'] != []:
                nouns.append(chunk)
        if len(nouns) > 1:
            for i in range(len(nouns)-1):
                for j in range(i+1,len(nouns)):
                    nX = nouns[i]
                    nY = nouns[j]
                    a.write(xy(nX, nY, sentence))

In [169]:
a=[[1,2],[3,4,5],[6,7]]
print([i for j in a for i in j])#クレイジー
print(sum(a, []))

[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]
